Import libraries

In [77]:
import pandas as pd
import numpy as np
import re
import time
import seaborn
import requests
import pytrends
from pytrends.request import TrendReq
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from urllib.parse import urlparse

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Cache-Control': 'no-cache',
    'Pragma': 'no-cache'
}

In [78]:
path = r'C:\Users\AshleyNguyen\Desktop\script\\'
df = pd.read_csv(path + 'zombie_companies_city.csv')

# Display the DataFrame
df

,prediction,InActive,LastFundingDate,LastFundingNo,TotalFunding,LastFundingAmount,CompanyName,Website,Description,CompanyOwnership,Industry Level 1,Industry Level 2,Industry Level 3,City,Country,FoundationYear
0,0.666624,True,2000-05-11 00:00:00,1.0,5.000000,5.000000,EasyChem ( Social/Platform Software),NaN,Provides of online services for business entit...,NaN,Information Technology,Software,Social/Platform Software,Sioux City,United States,2000.0
1,0.653978,True,2009-01-27 00:00:00,2.0,6.000000,6.000000,BrightView Systems,www.brightview-sys.com,Provider of a production technology for photov...,NaN,Energy,Energy Equipment,Alternative Energy Equipment,Petah Tikva,Israel,2008.0
2,0.642127,True,2006-12-18 00:00:00,2.0,8.500000,2.500000,Industrious Kid,www.industriouskid.com,Operator of a social networking site. The comp...,NaN,Information Technology,Software,Social/Platform Software,Oakland,United States,2006.0
3,0.640157,True,2013-01-01 00:00:00,1.0,5.000000,5.000000,Widbook,www.widbook.com,Developer of an online collaborative ebook com...,NaN,Information Technology,Software,Social/Platform Software,Sao Paulo,Brazil,2012.0
4,0.639256,True,2000-07-10 00:00:00,1.0,5.830000,5.830000,Cargobiz.com,www.cargobiz.com,Operator of business-to-business e-commerce co...,NaN,Information Technology,Software,Social/Platform Software,Hamburg,Germany,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35163,0.003425,False,2022-07-14 00:00:00,11.0,82.799928,39.999928,Mayvenn,shop.mayvenn.com,Retailer of online hair care products intended...,Privately Held (backing),Consumer Products and Services (B2C),Consumer Non-Durables,Personal Products,Emeryville,United States,2012.0
35164,0.003409,False,2022-02-02 00:00:00,11.0,90.334820,55.515246,Propel (Financial Software),www.joinpropel.com,Developer of a mobile finance platform designe...,Privately Held (backing),Information Technology,Software,Financial Software,New York,United States,2014.0
35165,0.003312,False,2022-08-01 00:00:00,11.0,377.920116,200.000000,Andela,www.andela.com,Operator of a global talent network intended t...,Privately Held (backing),Business Products and Services (B2B),Commercial Services,Human Capital Services,New York,United States,2014.0
35166,0.003263,False,2022-02-28 00:00:00,10.0,94.591082,50.000003,Qventus,www.qventus.com,Developer of hospital management software desi...,Privately Held (backing),Healthcare,Healthcare Technology Systems,Enterprise Systems (Healthcare),Mountain View,United States,2011.0


Generate combinations of company names

In [79]:
def generate_word_combinations(string):
    words = string.split()
    combinations = []

    # Generate combinations with n words
    for i in range(1, len(words) + 1):
        combination = ' '.join(words[:i])
        combinations.append(combination)

    return combinations

# Example
string = "Barking Cow Media LLC"
combinations = generate_word_combinations(string)
print(combinations)

['Barking', 'Barking Cow', 'Barking Cow Media', 'Barking Cow Media LLC']


Map city to state

In [80]:
def url_mapping(state):
    """
    Map the state with the corresponding business search API url

    Args:
        state (string): State where the company is founded and registered

    Returns:
        url (string): API url for the state.
    """
    if state == "California":
        url = "https://bizfileonline.sos.ca.gov/api/Records/businesssearch"
    elif state == "Montana":
        url = "https://biz.sosmt.gov/api/Records/businesssearch"
    elif state == "Idaho":
        url = "https://sosbiz.idaho.gov/api/Records/businesssearch"
    else:
        url = None
    return url

Function to get search results

In [81]:
def get_search_details(name, founding_year, state):
    url = url_mapping(state)
    headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9",
    "authorization": "undefined",
    "content-type": "application/json",
    "sec-ch-ua": "\"Google Chrome\";v=\"113\", \"Chromium\";v=\"113\", \"Not-A.Brand\";v=\"24\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin"
    }
    
    if state == "California":
        payload = {
            "SEARCH_VALUE": name,
            "SEARCH_FILTER_TYPE_ID": "0",
            "SEARCH_TYPE_ID": "1",
            "FILING_TYPE_ID": "",
            "STATUS_ID": "",
            "FILING_DATE": {"start": f'01/01/{founding_year}', "end": f'12/31/{founding_year}'},
            "CORPORATION_BANKRUPTCY_YN": False,
            "CORPORATION_LEGAL_PROCEEDINGS_YN": False
        }
    elif state == "Idaho":
        payload = {
            "SEARCH_VALUE": name,
            "QUERY_TYPE_ID": 1010,
            "FILING_TYPE_ID": "0",
            "FILING_SUBTYPE_ID": "0",
            "STATUS_ID": "0",
            "STATE": "",
            "COUNTY": "",
            "CRA_SEARCH_YN": False,
            "FILING_DATE": {"start": f'01/01/{founding_year}', "end": f'12/31/{founding_year}'},
            "EXPIRATION_DATE": {"start": None, "end": None}
        }
    
    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        data = response.json()
        num_result = len(list(data["rows"].keys()))
        if num_result == 0:
            company_id = None
        else: 
            company_id = list(data["rows"].keys())[0]
        print(num_result)
    else:
        print(response.status_code)
        data, num_result, company_id = None, None, None
        
    return data, num_result, company_id

In [82]:
### EXAMPLE TO CHECK IF FUNCTION IS RUNNING OK
url = url_mapping("Idaho")
name = "M2E Power"
founding_year = 2007

headers = {
"accept": "*/*",
"accept-language": "en-US,en;q=0.9",
"authorization": "undefined",
"content-type": "application/json",
"sec-ch-ua": "\"Google Chrome\";v=\"113\", \"Chromium\";v=\"113\", \"Not-A.Brand\";v=\"24\"",
"sec-ch-ua-mobile": "?0",
"sec-ch-ua-platform": "\"Windows\"",
"sec-fetch-dest": "empty",
"sec-fetch-mode": "cors",
"sec-fetch-site": "same-origin"
}

payload = {
    "SEARCH_VALUE": name,
    "QUERY_TYPE_ID": 1010,
    "FILING_TYPE_ID": "0",
    "FILING_SUBTYPE_ID": "0",
    "STATUS_ID": "0",
    "STATE": "",
    "COUNTY": "",
    "CRA_SEARCH_YN": False,
    "FILING_DATE": {"start": f'01/01/{founding_year}', "end": f'12/31/{founding_year}'},
    "EXPIRATION_DATE": {"start": None, "end": None}
}

response = requests.post(url, headers=headers, json=payload)

print(response)

if response.status_code == 200:
    data = response.json()
    num_result = len(list(data["rows"].keys()))
    if num_result == 0:
        company_id = None
    else: 
        company_id = list(data["rows"].keys())[0]
    print(num_result)
else:
    data, num_result, company_id = None, None, None

print(data)

<Response [200]>
1
{'template': [{'label': 'Form Info', 'id': 'TITLE'}, {'label': 'Status', 'id': 'STATUS'}, {'label': 'Filing Date', 'id': 'FILING_DATE'}, {'label': 'Agent', 'id': 'AGENT'}], 'rows': {'531136': {'SORT_INDEX': 0, 'TITLE': ['M2E POWER, INC. (531136)', 'Foreign Business Corporation'], 'ID': 531136, 'FILING_DATE': '10/17/2007', 'RECORD_NUM': '0000531136', 'AGENT': 'NO AGENT', 'STATUS': 'Inactive-Withdrawn', 'STANDING': 'Good Standing', 'ALERT': False, 'CAN_REINSTATE': False, 'CAN_FILE_AR': False, 'CAN_FILE_REINSTATEMENT': False}}}


Function to get company detail from its ID

In [83]:
def get_company_detail(data, company_id, state):
    
    if state == "California":
        company_url = f"https://bizfileonline.sos.ca.gov/api/FilingDetail/business/{company_id}/false"
    elif state == "Montana":
        company_url = f"https://biz.sosmt.gov/api/FilingDetail/business/{company_id}/false"
    elif state == "Idaho":
        company_url = f"https://sosbiz.idaho.gov/api/FilingDetail/business/{company_id}/false"
    else:
        company_url = None
    
    if len(list(data["rows"].keys())) >= 1:
        row = data["rows"][list(data["rows"].keys())[0]]
        search_detail = pd.DataFrame([row])

        company_id = list(data["rows"].keys())[0]

        response_company = requests.get(company_url)
        data_company = response_company.json()

        if 'message' in data_company:
            detail = pd.DataFrame
        else: 
            # Process the data as needed
            if state == "California":
                if len(pd.DataFrame(data_company['DRAWER_DETAIL_LIST']).columns) > 8:
                    extra_detail = pd.DataFrame(data_company['DRAWER_DETAIL_LIST']).iloc[[2, 3, 4, 5, 6, 7, 8, 11], :2].set_index('LABEL')
                else:
                    extra_detail = pd.DataFrame(data_company['DRAWER_DETAIL_LIST']).iloc[[2, 3, 4, 5, 6, 7], :2].set_index('LABEL')
            elif state == "Idaho":
                extra_detail = pd.DataFrame(data_company['DRAWER_DETAIL_LIST']).iloc[[0, 1, 2, 3, 4, 5], :2].set_index('LABEL')
            
            extra_detail = extra_detail.T.reset_index(drop=True)
            detail = pd.concat([search_detail, extra_detail], axis=1)
    else:
        detail = pd.DataFrame()
    return detail

Preprocess the csv (mapping city to state for the whole csv)

In [84]:
df_city = pd.read_csv(path+'us_cities_states_counties.csv', delimiter='|')

In [85]:
def map_city_to_state(city):
    # Filter the DataFrame for the given city
    filtered_df = df_city[df_city["City"] == city]

    # If the city is found in the DataFrame, return the corresponding state
    if not filtered_df.empty:
        state = filtered_df.iloc[0]["State full"]
        return state

    # If the city is not found, return None or any other desired value
    return None

In [86]:
df['State'] = df['City'].apply(map_city_to_state)

In [87]:
filtered = pd.read_csv(path+'zombie_a1234.csv')

In [88]:
filtered = df.merge(filtered[['CompanyName', 'NewDomain','keyword','InActive_A1','InActive_A2','average','InActive_A3']], on = 'CompanyName', how = 'right')

In [94]:
test = pd.concat([filtered[filtered['State'] == 'California'],filtered[filtered['State'] == 'Idaho']],axis=0)
test

,prediction,InActive,LastFundingDate,LastFundingNo,TotalFunding,LastFundingAmount,CompanyName,Website,Description,CompanyOwnership,...,City,Country,FoundationYear,State,NewDomain,keyword,InActive_A1,InActive_A2,average,InActive_A3
2,0.517180,False,2021-09-20 00:00:00,1.0,7.500000,7.500000,Stageverse,www.stageverse.com,Developer of social metaverse platform designe...,NaN,...,Los Angeles,United States,2017.0,California,stageverse,stageverse,False,False,6.461538,False
3,0.515617,False,2015-06-08 00:00:00,3.0,8.190000,0.150000,MobSoc Media,www.mobsocmedia.com,Developer of social applications for creating ...,NaN,...,San Francisco,United States,2012.0,California,mobsocmedia,mobsocmedia,False,False,0.000000,False
5,0.498849,False,2011-08-05 00:00:00,1.0,5.800000,5.800000,Quark Games,www.quarkgames.com,Developer of web based and mobile games. The c...,NaN,...,Cupertino,United States,2008.0,California,quarkgames,quarkgames,False,False,6.653846,False
11,0.465889,False,2007-09-05 00:00:00,1.0,9.460000,9.460000,Vivaty,www.vivaty.com,Developer of 3D social games and application p...,NaN,...,San Francisco,United States,2007.0,California,vivaty,vivaty,False,False,7.865385,False
13,0.463630,False,2001-04-01 00:00:00,1.0,8.000000,8.000000,oneREV,www.onerev.com,Developer of supply management software. The c...,NaN,...,Cupertino,United States,2000.0,California,onerev,onerev,False,False,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,0.173712,False,2007-11-16 00:00:00,2.0,8.000000,8.000000,M2E Power,www.m2epower.com,Developer of batteries to generate and store p...,NaN,...,Boise,United States,2007.0,Idaho,m2epower,m2epower,False,False,0.000000,False
1728,0.079719,False,2021-08-01 00:00:00,3.0,13.200000,4.900000,Crave Kitchens,www.cravekitchens.io,Operator of restaurants intended to offer a de...,Privately Held (backing),...,Boise,United States,2019.0,Idaho,cravekitchens,cravekitchens,False,False,0.000000,False
2580,0.055102,False,2020-04-11 00:00:00,3.0,12.999874,2.553000,Empowered Education,www.empowered-edu.com,Operator of an online educational platform des...,Privately Held (backing),...,Boise,United States,2015.0,Idaho,empowered-edu,empowered-edu,False,False,0.000000,False
3002,0.046157,False,2016-01-11 00:00:00,7.0,8.310000,4.000000,Wevorce,www.wevorce.com,Provider of divorce handling services intended...,NaN,...,Boise,United States,2012.0,Idaho,wevorce,wevorce,False,False,9.941176,False


In [95]:
def are_all_elements_none(lst):
    check = all(element is None or element == 0 for element in lst)
    return check

In [96]:
pos = 0
num = len(test['CompanyName']) - pos
list_companies = test['CompanyName'].tolist()[pos:]
list_years = test['FoundationYear'].tolist()[pos:]
list_state = test['State'].tolist()[pos:]

import math
for i in range(len(list_years)):
    if math.isnan(list_years[i]):
        list_years[i] = 0

list_years = list(map(int, list_years))


In [97]:
final = pd.DataFrame()

for i in range(num):
    name = re.sub(r'\(.*?\)', '', list_companies[i])
    founding_year = list_years[i]
    state = list_state[i]
    # Generate different combinations of name to try for the search
    name_combinations = generate_word_combinations(name)
    print(name_combinations)
    # Collect the number of search result, data and id of the first company in the search result
    data, num_res, company_id = [],[],[]
    for i in range(len(name_combinations)):
        text = name_combinations[i]
        d, n, id = get_search_details(text, founding_year, state)
        data.append(d)
        num_res.append(n)
        company_id.append(id)

    # Use the search query with the smallest number search results returned (ideally 1)
    if are_all_elements_none(num_res) or num_res[0] == 0:
        company_tax_detail = pd.DataFrame()
    else:
        min_index = np.nonzero(num_res)[0].min()
        company_tax_detail = get_company_detail(data[min_index], company_id[min_index], state)
        company_tax_detail['NumResult'] = num_res[min_index]
        company_tax_detail['CompanyName'] = name
        print(company_tax_detail)
    final = pd.concat([final,company_tax_detail], axis=0)

final

['Stageverse']
0
['MobSoc', 'MobSoc Media']
1
1
   SORT_INDEX                              TITLE       ID FILING_DATE  \
0           0  [MOBSOC MEDIA LLC (201228210247)]  6732454  09/26/2012   

     RECORD_NUM   FORMED_IN       AGENT  STATUS  \
0  201228210247  CALIFORNIA  LARRY KANE  Active   

                      ENTITY_TYPE       STANDING  ...  CAN_FILE_AR  \
0  Limited Liability Company - CA  Good Standing  ...         True   

   CAN_FILE_REINSTATEMENT  Standing - SOS  Standing - FTB Standing - Agent  \
0                   False            Good            Good             Good   

  Standing - VCFCF   Formed In                     Entity Type NumResult  \
0             Good  CALIFORNIA  Limited Liability Company - CA         1   

    CompanyName  
0  MobSoc Media  

[1 rows x 22 columns]
['Quark', 'Quark Games']
4
1
   SORT_INDEX                          TITLE       ID FILING_DATE RECORD_NUM  \
0           0  [QUARK GAMES, INC. (3182829)]  1221548  12/30/2008    3182829   

  

,SORT_INDEX,TITLE,ID,FILING_DATE,RECORD_NUM,FORMED_IN,AGENT,STATUS,ENTITY_TYPE,STANDING,...,Entity Type,NumResult,CompanyName,Inactive Date,Foreign Name,Principal Address,Mailing Address,Filing Type,Status,Term of Duration
0,0,[MOBSOC MEDIA LLC (201228210247)],6732454,09/26/2012,201228210247,CALIFORNIA,LARRY KANE,Active,Limited Liability Company - CA,Good Standing,...,Limited Liability Company - CA,1,MobSoc Media,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,"[QUARK GAMES, INC. (3182829)]",1221548,12/30/2008,3182829,DELAWARE,JAMES LUO,Terminated,Stock Corporation - Out of State - Stock,Good Standing,...,NaN,4,Quark Games,12/15/2016,NaN,NaN,NaN,NaN,NaN,NaN
0,0,"[VIVATY, INC. (3007215)]",2170679,07/18/2007,3007215,DELAWARE,LIOR NUCHI,Terminated,Stock Corporation - Out of State - Stock,Good Standing,...,NaN,1,Vivaty,08/31/2010,NaN,NaN,NaN,NaN,NaN,NaN
0,0,"[ONEREV, INC. (2266627)]",3924071,10/27/2000,2266627,DELAWARE,NAGEEN SHARMA,Forfeited - FTB/SOS,Stock Corporation - Out of State - Stock,Not Good Standing,...,NaN,1,oneREV,06/09/2005,NaN,NaN,NaN,NaN,NaN,NaN
0,0,"['' N & J FASHION, INC.'' (3245690)]",3391644,04/27/2010,3245690,CALIFORNIA,GARNIK NIKOGHOSYAN,Active,Stock Corporation - CA - General,Good Standing,...,Stock Corporation - CA - General,319,Fashion One,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,[MIRO EXPORTS LLC (201114110286)],5747011,03/15/2011,201114110286,CALIFORNIA,MIRAN EMRICH,Terminated,Limited Liability Company - CA,Good Standing,...,NaN,3,Miro,12/05/2011,NaN,NaN,NaN,NaN,NaN,NaN
0,0,"[M2E POWER, INC. (531136), Foreign Business Co...",531136,10/17/2007,0000531136,NaN,NO AGENT,Inactive-Withdrawn,NaN,Good Standing,...,NaN,1,M2E Power,NaN,"M2E POWER, INC.","C/O HIGHWAY 12 VENTURES 802 W. BANNOCK, 7TH FL...",NaN,Foreign Business Corporation,Inactive-Withdrawn,Perpetual
0,0,"[Crave Air Charter, LLC (3659236), Limited Lia...",686162,10/31/2019,0003659236,NaN,Scot Eastman,Inactive-Dissolved (Administrative),NaN,Not Good Standing,...,NaN,3,Crave Kitchens,NaN,None,"SCOT EASTMAN\n330 FOREST STREET\nMCCALL, ID 83638",NaN,Limited Liability Company (D),Inactive-Dissolved (Administrative),Perpetual
0,0,"[EMPOWERED SOLAR (538904), Assumed Business Name]",538904,04/06/2015,0000538904,NaN,NO AGENT,Active-Current,NaN,Good Standing,...,NaN,3,Empowered Education,NaN,None,N/A,NaN,Assumed Business Name,Active-Current,Perpetual


In [98]:
final.to_csv(path+'tax_scraping.csv')